## Lista de eliminaciones de columnas de la BDD 'Diseño de registros de la base hogar'.
* AGLOMERADO(31): Solo nos quedaremos con estas filas, porque trabajaremos solo registros de TDF.
* REALIZADA: No aporta a la nada a la base de datos
* ANO4: Como toda la BDD es del año 2018, podems eliminarla.
* TRIMESTRE: Como toda la BDD es del mismo trimestre, podems eliminarla.
* POND_TIC: No hay suficientes datos sobre esta columna.

## Lista de eliminaciones de columnas de la BDD 'Diseño de registros de la base personas'.
* AGLOMERADO(31): Solo nos quedaremos con estas filas, porque trabajaremos solo registros de TDF.
* ANO4: Como toda la BDD es del año 2018, podems eliminarla.
* TRIMESTRE: Como toda la BDD es del mismo trimestre, podems eliminarla.
* POND_TIC: No hay suficientes datos sobre esta columna.